In [169]:
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import load_img,img_to_array
from sklearn.metrics import mean_squared_error
from keras.initializers import RandomNormal
from keras.applications.vgg16 import VGG16
from keras.optimizers import SGD
from keras.models import Model,Sequential
from keras.layers import *
from keras import backend as K
from keras.models import model_from_json
from matplotlib import cm as CM
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
import scipy.io as io
from PIL import Image
import PIL
import h5py
import os
import glob
import cv2
import random
import math
import sys
import numpy as np

In [2]:
!pip install opencv-python

In [170]:
K.clear_session()
root = 'data'

In [171]:
part_A_train = os.path.join(root,'part_A_final/train_data','images')
part_A_test = os.path.join(root,'part_A_final/test_data','images')
part_B_train = os.path.join(root,'part_B_final/train_data','images')
part_B_test = os.path.join(root,'part_B_final/test_data','images')
temp = 'test_images'
path_sets = [part_A_train,part_B_train]

In [172]:
img_paths = []

for path in path_sets:
    
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        
        img_paths.append(str(img_path))
        
print("Total images : ",len(img_paths))

Total images :  700


In [173]:
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)
print(len(img_paths))

700


In [227]:
def create_img(path):
    #Function to load,normalize and return image 
    im = Image.open(path).convert('RGB')
    
    im = np.array(im)
    
    im = im/255.0
    
    im[:,:,0]=(im[:,:,0]-0.485)/0.229
    im[:,:,1]=(im[:,:,1]-0.456)/0.224
    im[:,:,2]=(im[:,:,2]-0.406)/0.225

    print(im.shape)
   # im = np.expand_dims(im,axis  = 0)
    print(im.shape)
    
    return im

def get_input(path):
    path = path[0] 
    img = create_img(path)
    return(img)
    
    
    
def get_output(path):
    #import target
    #resize target
    
    gt_file = h5py.File(path,'r')
    
    target = np.asarray(gt_file['density'])
    
    img = cv2.resize(target,(int(target.shape[1]/8),int(target.shape[0]/8)),interpolation = cv2.INTER_CUBIC)*64
    type(img)
    img = np.expand_dims(img,axis  = 2)       #default 3

    print(img.shape)
    
    return img
    
    
    
def preprocess_input(image,target):
    #crop image
    #crop target
    #resize target
    crop_size = (int(image.shape[0]/2),int(image.shape[1]/2))
    
    
    if random.randint(0,9)<= -1:            
            dx = int(random.randint(0,1)*image.shape[0]*1./2)
            dy = int(random.randint(0,1)*image.shape[1]*1./2)
    else:
            dx = int(random.random()*image.shape[0]*1./2)
            dy = int(random.random()*image.shape[1]*1./2)

    #print(crop_size , dx , dy)
    img = image[dx : crop_size[0]+dx , dy:crop_size[1]+dy]
    
    target_aug = target[dx:crop_size[0]+dx,dy:crop_size[1]+dy]
    print(img.shape)

    return(img,target_aug)
    
   

In [215]:
#Image data generator 
def image_generator(files, batch_size = 64):
    
    while True:
        
        input_path = np.random.choice(a = files, size = batch_size)
        
        
        batch_input = []
        batch_output = [] 
          
        #for input_path in batch_paths:
        
        inputt = get_input(input_path )
        
        output = get_output(input_path[0].replace('.jpg','.h5').replace('images','ground') )
            
       
        batch_input += [inputt]
        batch_output += [output]
    

        batch_x = np.array( batch_input )
        batch_y = np.array( batch_output )
        
        yield( batch_x, batch_y )


In [99]:
#############################

In [216]:
def save_mod(model , str1 , str2):
    model.save_weights(str1)
    
    model_json = model.to_json()
    
    with open(str2, "w") as json_file:
        json_file.write(model_json)

In [217]:
def init_weights_vgg(model):
    #vgg =  VGG16(weights='imagenet', include_top=False)
    
    json_file = open('models/VGG_16.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights("weights/VGG_16.h5")
    
    vgg = loaded_model
    
    vgg_weights=[]                         
    for layer in vgg.layers:
        if('conv' in layer.name):
            vgg_weights.append(layer.get_weights())
    
    
    offset=0
    i=0
    while(i<10):
        if('conv' in model.layers[i+offset].name):
            model.layers[i+offset].set_weights(vgg_weights[i])
            i=i+1
            #print('h')
            
        else:
            offset=offset+1

    return (model)
    

In [218]:
def euclidean_distance_loss(y_true, y_pred):
    # Euclidean distance as a measure of loss (Loss function) 
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))


In [ ]:
#             else:
#                 model.add(Conv2D(64, kernel_size = kernel,activation = 'relu', padding='same',input_shape = (rows, cols, 3), kernel_initializer = init))
#                 model.add(Conv2D(64, kernel_size = kernel,activation = 'relu', padding='same', kernel_initializer = init))
#                 model.add(MaxPooling2D(strides=2))
#                 model.add(Conv2D(128,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
#                 model.add(Conv2D(128,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
#                 model.add(MaxPooling2D(strides=2))
#                 model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
#                 model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
#                 model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
#                 model.add(MaxPooling2D(strides=2))            
#                 model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same', kernel_initializer = init))
#                 model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same', kernel_initializer = init))
#                 model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same', kernel_initializer = init))
                

In [228]:
# Neural network model : VGG + Conv
def CrowdNet():  
            #Variable Input Size
            rows = None
            cols = None
            
            #Batch Normalisation option
            
            batch_norm = 0      #default 0 ---sudeep
            kernel = (3, 3)
            init = RandomNormal(stddev=0.01)
            model = Sequential() 
            
            #custom VGG:
            
            if(batch_norm):
                model.add(Conv2D(64, kernel_size = kernel, input_shape = (rows,cols,3),activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(64, kernel_size = kernel,activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(MaxPooling2D(strides=2))
                model.add(Conv2D(128,kernel_size = kernel, activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(128,kernel_size = kernel, activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(MaxPooling2D(strides=2))
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(MaxPooling2D(strides=2))            
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same'))
                model.add(BatchNormalization())

                
            #Conv2D          ----------by sudeep 2nd field (2,2) works
            model.add(Conv2D(512, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(512, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(512, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(256, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(128, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(64, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(1, (1, 1), dilation_rate = 1, kernel_initializer = init, padding = 'same'))
           
            
            sgd = SGD(lr = 1e-7, decay = (5*1e-4), momentum = 0.95)
            model.compile(optimizer=sgd, loss=euclidean_distance_loss, metrics=['mse'])
            
            model = init_weights_vgg(model)
            
            return model

In [229]:
model = CrowdNet()

In [230]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_35 (Conv2D)           (None, None, None, 64)    1792      
_________________________________________________________________
conv2d_36 (Conv2D)           (None, None, None, 64)    36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, None, None, 64)    0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, None, None, 128)   73856     
_________________________________________________________________
conv2d_38 (Conv2D)           (None, None, None, 128)   147584    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, None, None, 128)   0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, None, None, 256)  

In [231]:
train_gen = image_generator(img_paths,1)

In [232]:
img_paths

['data\\part_A_final/test_data\\images\\IMG_1.jpg',
 'data\\part_A_final/test_data\\images\\IMG_10.jpg',
 'data\\part_A_final/test_data\\images\\IMG_100.jpg',
 'data\\part_A_final/test_data\\images\\IMG_101.jpg',
 'data\\part_A_final/test_data\\images\\IMG_102.jpg',
 'data\\part_A_final/test_data\\images\\IMG_103.jpg',
 'data\\part_A_final/test_data\\images\\IMG_104.jpg',
 'data\\part_A_final/test_data\\images\\IMG_105.jpg',
 'data\\part_A_final/test_data\\images\\IMG_106.jpg',
 'data\\part_A_final/test_data\\images\\IMG_107.jpg',
 'data\\part_A_final/test_data\\images\\IMG_108.jpg',
 'data\\part_A_final/test_data\\images\\IMG_109.jpg',
 'data\\part_A_final/test_data\\images\\IMG_11.jpg',
 'data\\part_A_final/test_data\\images\\IMG_110.jpg',
 'data\\part_A_final/test_data\\images\\IMG_111.jpg',
 'data\\part_A_final/test_data\\images\\IMG_112.jpg',
 'data\\part_A_final/test_data\\images\\IMG_113.jpg',
 'data\\part_A_final/test_data\\images\\IMG_114.jpg',
 'data\\part_A_final/test_data\\

In [233]:
sgd = SGD(lr = 1e-7, decay = (5*1e-4), momentum = 0.95)
model.compile(optimizer=sgd, loss=euclidean_distance_loss, metrics=['accuracy'])

In [26]:
model.fit_generator(train_gen,epochs=3, steps_per_epoch=700 , verbose=True,max_queue_size=1)

NameError: name 'model' is not defined

In [186]:
save_mod(model,"weights/model_A_weights.h5","models/Model.json")

# Analysis of the model

In [1]:
from keras.models import model_from_json
import os
import cv2
import glob
import h5py
import pandas as pd
from sklearn.metrics import mean_absolute_error
import scipy.io as io
from PIL import Image
import numpy as np

Using TensorFlow backend.


In [10]:
def load_model():
    
    json_file = open('models/Model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights("weights/model_A_weights.h5")
    return loaded_model


def create_img(path):
    im = Image.open(path).convert('RGB')
    
    im = np.array(im)
    
    im = im/255.0
    
    im[:,:,0]=(im[:,:,0]-0.485)/0.229
    im[:,:,1]=(im[:,:,1]-0.456)/0.224
    im[:,:,2]=(im[:,:,2]-0.406)/0.225


    im = np.expand_dims(im,axis  = 0)
    return im

In [3]:
root = 'data'

In [5]:

part_A_train = os.path.join(root,'part_A_final/train_data','images')
part_A_test = os.path.join(root,'part_A_final/test_data','images')
part_B_train = os.path.join(root,'part_B_final/train_data','images')
part_B_test = os.path.join(root,'part_B_final/test_data','images')
path_sets = [part_B_test] # to test on full test dataset
# path_sets = [part_A_train,part_B_train]

In [6]:

img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)
print(len(img_paths))

316


In [7]:
print(len(img_paths))

316


In [11]:
model_analysis = load_model()
name = []
y_true = []
y_pred = []


In [13]:
model_analysis.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, None, None, 64)    1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, None, None, 64)    0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 128)   73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, None, None, 128)   147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, None, None, 128)   0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, None, None, 256)  

In [15]:

for image in img_paths:
    name.append(image)
    gt = h5py.File(image.replace('.jpg','.h5').replace('images','ground') )
    groundtruth = np.asarray(gt['density'])
    num1 = np.sum(groundtruth)
    y_true.append(np.sum(num1))
    img = create_img(image)
    #print(img)
    # Getting count
    image_sum = model_analysis.predict(img)
    print(image_sum)
    num = np.sum(image_sum)
#    print(num)
    y_pred.append(np.sum(num))

    
data = pd.DataFrame({'name': name,'y_pred': y_pred,'y_true': y_true})
data.to_csv('CSV/A_on_A_test.csv', sep=',')

C:\Users\sudee\anaconda3\lib\site-packages\ipykernel_launcher.py:3: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  This is separate from the ipykernel package so we can avoid doing imports until


[[[[0.00070822]
   [0.00082004]
   [0.00067159]
   ...
   [0.00070166]
   [0.00071382]
   [0.00087435]]

  [[0.00074796]
   [0.00063897]
   [0.00076423]
   ...
   [0.00055305]
   [0.00073064]
   [0.00064781]]

  [[0.00103571]
   [0.00097153]
   [0.00083712]
   ...
   [0.00052565]
   [0.00141307]
   [0.00079544]]

  ...

  [[0.00078803]
   [0.00075112]
   [0.00098795]
   ...
   [0.00045817]
   [0.00075038]
   [0.00065024]]

  [[0.00093786]
   [0.00089875]
   [0.00087542]
   ...
   [0.00064451]
   [0.00100869]
   [0.00098201]]

  [[0.00095039]
   [0.00098912]
   [0.00089292]
   ...
   [0.00065456]
   [0.00073388]
   [0.00094353]]]]
[[[[0.00077779]
   [0.0011451 ]
   [0.00098934]
   ...
   [0.0006861 ]
   [0.00072916]
   [0.00080992]]

  [[0.00064169]
   [0.0009522 ]
   [0.00063175]
   ...
   [0.00077618]
   [0.00092218]
   [0.0008392 ]]

  [[0.00053585]
   [0.00085097]
   [0.00046159]
   ...
   [0.00067248]
   [0.00167414]
   [0.00096005]]

  ...

  [[0.00053665]
   [0.00018981]
   [0.00

[[[[0.00086838]
   [0.00098413]
   [0.00100507]
   ...
   [0.00081467]
   [0.00072667]
   [0.00082529]]

  [[0.00093368]
   [0.00089741]
   [0.00099909]
   ...
   [0.0007853 ]
   [0.00082623]
   [0.00090708]]

  [[0.00091229]
   [0.00123163]
   [0.00105919]
   ...
   [0.00072078]
   [0.00085417]
   [0.00103047]]

  ...

  [[0.00092284]
   [0.00091488]
   [0.00065489]
   ...
   [0.00084667]
   [0.00088958]
   [0.00095675]]

  [[0.00097474]
   [0.00090402]
   [0.00107042]
   ...
   [0.00094348]
   [0.00089504]
   [0.00079899]]

  [[0.00061399]
   [0.00088637]
   [0.00078474]
   ...
   [0.00068439]
   [0.00088487]
   [0.00098574]]]]
[[[[0.00090086]
   [0.0009334 ]
   [0.00097311]
   ...
   [0.00103815]
   [0.00092037]
   [0.00089981]]

  [[0.00088793]
   [0.00079159]
   [0.00085171]
   ...
   [0.00068437]
   [0.00069729]
   [0.00075019]]

  [[0.00082368]
   [0.00090303]
   [0.00099989]
   ...
   [0.00067483]
   [0.00124717]
   [0.000903  ]]

  ...

  [[0.00056531]
   [0.00066649]
   [0.00

[[[[0.00073686]
   [0.00083878]
   [0.00112704]
   ...
   [0.00076736]
   [0.00074635]
   [0.0008781 ]]

  [[0.00057955]
   [0.00062916]
   [0.00086989]
   ...
   [0.00079926]
   [0.00078001]
   [0.0009414 ]]

  [[0.00105753]
   [0.00094174]
   [0.00041449]
   ...
   [0.00027491]
   [0.0005727 ]
   [0.00094442]]

  ...

  [[0.00115008]
   [0.0008391 ]
   [0.00070782]
   ...
   [0.00046014]
   [0.00078913]
   [0.00097676]]

  [[0.00104373]
   [0.00093641]
   [0.0005639 ]
   ...
   [0.00056725]
   [0.00078889]
   [0.00106745]]

  [[0.00112915]
   [0.00082978]
   [0.0002463 ]
   ...
   [0.00058642]
   [0.00078228]
   [0.00084686]]]]
[[[[9.8295847e-04]
   [1.0109524e-03]
   [9.6112874e-04]
   ...
   [1.1521039e-03]
   [6.7814288e-04]
   [5.1587616e-04]]

  [[9.4627216e-04]
   [9.1442658e-04]
   [7.8025961e-04]
   ...
   [8.9760160e-04]
   [7.3325349e-04]
   [7.7155471e-04]]

  [[1.0169072e-03]
   [1.2153953e-03]
   [5.6067976e-04]
   ...
   [4.5486318e-05]
   [5.7828880e-04]
   [1.1933249e

[[[[0.00077725]
   [0.00073965]
   [0.00057369]
   ...
   [0.0007039 ]
   [0.00076045]
   [0.00097658]]

  [[0.00088828]
   [0.00066415]
   [0.00104314]
   ...
   [0.00066729]
   [0.00067838]
   [0.00103008]]

  [[0.00090359]
   [0.0009118 ]
   [0.00089883]
   ...
   [0.00042962]
   [0.00093433]
   [0.00126234]]

  ...

  [[0.00040896]
   [0.00034874]
   [0.00027746]
   ...
   [0.00091101]
   [0.00090981]
   [0.0010932 ]]

  [[0.00100582]
   [0.00081563]
   [0.00059224]
   ...
   [0.00062034]
   [0.0009049 ]
   [0.00098909]]

  [[0.00098245]
   [0.00085283]
   [0.00063929]
   ...
   [0.00068182]
   [0.00112092]
   [0.00083911]]]]
[[[[0.00075139]
   [0.00078539]
   [0.00054858]
   ...
   [0.00087498]
   [0.00065107]
   [0.0006775 ]]

  [[0.00066905]
   [0.00073425]
   [0.00046592]
   ...
   [0.00091432]
   [0.00066104]
   [0.0005393 ]]

  [[0.00073789]
   [0.00064462]
   [0.00034473]
   ...
   [0.00082145]
   [0.00137594]
   [0.0008415 ]]

  ...

  [[0.00080169]
   [0.00084581]
   [0.00

...

[[[[0.00043329]
   [0.00061941]
   [0.00057739]
   ...
   [0.0009589 ]
   [0.00092928]
   [0.0008682 ]]

  [[0.00060157]
   [0.00066112]
   [0.00076013]
   ...
   [0.00107024]
   [0.00081316]
   [0.00089479]]

  [[0.00071489]
   [0.00089472]
   [0.00074218]
   ...
   [0.00067462]
   [0.0010308 ]
   [0.00112589]]

  ...

  [[0.00058041]
   [0.00079178]
   [0.00075573]
   ...
   [0.00036944]
   [0.00054405]
   [0.00040576]]

  [[0.0008833 ]
   [0.00090583]
   [0.00077256]
   ...
   [0.00025338]
   [0.00091082]
   [0.00078121]]

  [[0.00097271]
   [0.00098935]
   [0.00092453]
   ...
   [0.00035985]
   [0.00089564]
   [0.00087845]]]]
[[[[0.00098085]
   [0.00075156]
   [0.00042167]
   ...
   [0.00082705]
   [0.00077592]
   [0.00091081]]

  [[0.00068987]
   [0.00077487]
   [0.00063729]
   ...
   [0.00064034]
   [0.00071227]
   [0.00099479]]

  [[0.00076809]
   [0.0009005 ]
   [0.00056038]
   ...
   [0.00086891]
   [0.00102921]
   [0.00106213]]

  ...

  [[0.00078153]
   [0.00065638]
   [0.00

[[[[0.00090801]
   [0.0006886 ]
   [0.0008439 ]
   ...
   [0.00108237]
   [0.00072287]
   [0.0007701 ]]

  [[0.00072118]
   [0.00068803]
   [0.00050178]
   ...
   [0.00082566]
   [0.00069438]
   [0.00071586]]

  [[0.0008369 ]
   [0.00064368]
   [0.00046889]
   ...
   [0.00074566]
   [0.00085666]
   [0.00076786]]

  ...

  [[0.00084002]
   [0.00080453]
   [0.00096431]
   ...
   [0.00057052]
   [0.00065587]
   [0.00078671]]

  [[0.00089806]
   [0.00081497]
   [0.00093136]
   ...
   [0.00084297]
   [0.00087523]
   [0.0008121 ]]

  [[0.00094023]
   [0.0008881 ]
   [0.00100546]
   ...
   [0.00069712]
   [0.00088023]
   [0.0009127 ]]]]
[[[[ 7.05666665e-04]
   [ 8.32309481e-04]
   [ 8.45783856e-04]
   ...
   [ 9.84942075e-04]
   [ 8.33822240e-04]
   [ 7.22151482e-04]]

  [[ 7.65980920e-04]
   [ 1.00983493e-03]
   [ 7.38106843e-04]
   ...
   [ 9.59600380e-04]
   [ 7.63182063e-04]
   [ 7.92920589e-04]]

  [[ 9.67128784e-04]
   [ 9.69112851e-04]
   [ 1.08430302e-03]
   ...
   [ 1.08620292e-03]
 

[[[[0.00070343]
   [0.00064268]
   [0.00074533]
   ...
   [0.00011524]
   [0.00087043]
   [0.00085011]]

  [[0.00061678]
   [0.00076214]
   [0.00068848]
   ...
   [0.00064558]
   [0.00069114]
   [0.0008539 ]]

  [[0.00110219]
   [0.00125096]
   [0.00054457]
   ...
   [0.00043586]
   [0.00137286]
   [0.00070395]]

  ...

  [[0.00089359]
   [0.00092924]
   [0.00056392]
   ...
   [0.00053941]
   [0.00074914]
   [0.00073835]]

  [[0.00093624]
   [0.00084158]
   [0.00075216]
   ...
   [0.00066318]
   [0.00107971]
   [0.00083067]]

  [[0.00094653]
   [0.00092973]
   [0.00066893]
   ...
   [0.00037997]
   [0.00085851]
   [0.00087047]]]]
[[[[0.00071372]
   [0.00079644]
   [0.00068555]
   ...
   [0.00064641]
   [0.00067391]
   [0.0011855 ]]

  [[0.00070216]
   [0.00072987]
   [0.0008599 ]
   ...
   [0.00086124]
   [0.00076614]
   [0.0009905 ]]

  [[0.00080484]
   [0.00081449]
   [0.00044684]
   ...
   [0.00054359]
   [0.00106263]
   [0.00111043]]

  ...

  [[0.00086537]
   [0.00083724]
   [0.00

[[[[0.00092871]
   [0.0009936 ]
   [0.0009987 ]
   ...
   [0.0010647 ]
   [0.00095405]
   [0.00065181]]

  [[0.000828  ]
   [0.00094875]
   [0.00091432]
   ...
   [0.00092912]
   [0.00107487]
   [0.0003777 ]]

  [[0.0007571 ]
   [0.00090405]
   [0.0009158 ]
   ...
   [0.00066936]
   [0.00102608]
   [0.00064736]]

  ...

  [[0.0008964 ]
   [0.00081447]
   [0.00081975]
   ...
   [0.00070672]
   [0.00085179]
   [0.00079412]]

  [[0.00100536]
   [0.00086405]
   [0.00084043]
   ...
   [0.00060561]
   [0.0008822 ]
   [0.00080112]]

  [[0.00093291]
   [0.00098098]
   [0.00094118]
   ...
   [0.00065387]
   [0.00080925]
   [0.00093327]]]]
[[[[8.5975189e-04]
   [9.4037375e-04]
   [5.7982333e-04]
   ...
   [9.2626957e-04]
   [6.6229806e-04]
   [6.1140949e-04]]

  [[6.4526097e-04]
   [6.8176095e-04]
   [5.5031030e-04]
   ...
   [9.0184517e-04]
   [9.6640451e-04]
   [8.8322279e-04]]

  [[5.4443278e-04]
   [6.1901199e-04]
   [9.0291194e-04]
   ...
   [2.9581191e-05]
   [7.1774150e-04]
   [1.0390593e

ValueError: arrays must all be same length

In [17]:
data = pd.read_csv('CSV/A_on_A_test.csv')
y_true = data['y_true']
y_pred = data['y_pred']

In [18]:
data.head()

,Unnamed: 0,name,y_pred,y_true
0,0,data\part_A_final/test_data\images\IMG_1.jpg,4.085224,171.829422
1,1,data\part_A_final/test_data\images\IMG_10.jpg,4.769192,497.063629
2,2,data\part_A_final/test_data\images\IMG_100.jpg,1.840265,383.926056
3,3,data\part_A_final/test_data\images\IMG_101.jpg,3.703334,210.397263
4,4,data\part_A_final/test_data\images\IMG_102.jpg,0.699472,217.376938


In [21]:
ans = mean_absolute_error(np.array(y_true),np.array(y_pred))

In [22]:
print("MAE : " , ans )

MAE :  426.10253292977154


In [207]:
data = pd.read_csv('CSV/B_on_B_test.csv' , sep='\t')
y_true = data['y_true']
y_pred = data['y_pred']

In [208]:
ans = mean_absolute_error(np.array(y_true),np.array(y_pred))

In [209]:
print("MAE : " , ans )

MAE :  11.001472832281378


In [25]:
t

AttributeError: 'Sequential' object has no attribute 'history'